### Lease

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL ,lang='english',check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.42.12

#### Simple Lease

Each rental on all periods are same during life time of lease

In [37]:
lease = ["Lease"
            ,{"fixRental": 12.0
            ,"originTerm": 6
            ,"freq": ["DayOfMonth",15]
            ,"originDate": "2022-01-05"
            ,"status":"Current"
            ,"remainTerm": 6}]

r = localAPI.runAsset("2022-01-02"
                     ,[lease]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)
r[0]

,Balance,Rental
Date,,
2022-01-15,1812,120
2022-02-15,1440,372
2022-03-15,1104,336
2022-04-15,732,372
2022-05-15,372,360
2022-06-15,0,372


With aging lease ( remain term = 3 )

In [39]:
lease = ["Lease"
            ,{"fixRental": 12.0
            ,"originTerm": 6
            ,"freq": ["DayOfMonth",15]
            ,"originDate": "2022-01-05"
            ,"status":"Current"
            ,"remainTerm": 3}]

r = localAPI.runAsset("2022-01-02"
                     ,[lease]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)
r[0]

,Balance,Rental
Date,,
2022-04-15,732,372
2022-05-15,372,360
2022-06-15,0,372


#### Step Up Lease

The payment amount changes during the lease term. It can be :
* increase/decrease by a constant rate by each period
* increase/decrease by a constant amount by each period
* increase/decrease by an amount vector in all periods
* increase/decrease by an rate vector in all periods

##### Increase/decrease by constant rate

On each payment period, rental increase by 10%

In [18]:
leaseStepUp = ["Lease"
                ,{"initRental": 12.0
                ,"stepUp":("flatRate",1.1)
                ,"originTerm": 4
                ,"freq": ["DayOfMonth",15]
                ,"originDate": "2022-01-05"
                ,"status":"Current"
                ,"remainTerm": 4}
              ]
r = localAPI.runAsset("2022-01-02"
                     ,[leaseStepUp]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)

r[0]

,Balance,Rental
Date,,
2022-01-15,1310.83,120.00
2022-02-15,901.63,409.20
2022-03-15,495.07,406.56
2022-04-15,0.00,495.07


On each payment period, rental increase by 1$

In [19]:
leaseStepUp = ["Lease"
                ,{"initRental": 12.0
                ,"stepUp":("flatAmount",1.0)
                ,"originTerm": 6
                ,"freq": ["DayOfMonth",15]
                ,"originDate": "2022-01-05"
                ,"status":"Current"
                ,"remainTerm": 6}
              ]
r = localAPI.runAsset("2022-01-02"
                     ,[leaseStepUp]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)

r[0]

,Balance,Rental
Date,,
2022-01-15,2267,120
2022-02-15,1864,403
2022-03-15,1472,392
2022-04-15,1007,465
2022-05-15,527,480
2022-06-15,0,527


User can set rental change by rate on each payment period

In [26]:
leaseStepUp = ["Lease"
                ,{"initRental": 12.0
                ,"stepUp":("byRates",1.1,1.0,1.0,1.0,1.2)
                ,"originTerm": 6
                ,"freq": ["DayOfMonth",15]
                ,"originDate": "2022-01-05"
                ,"status":"Current"
                ,"remainTerm": 6}
              ]
r = localAPI.runAsset("2022-01-02"
                     ,[leaseStepUp]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)

r[0]

,Balance,Rental
Date,,
2022-01-15,2075.04,120.00
2022-02-15,1665.84,409.20
2022-03-15,1296.24,369.60
2022-04-15,887.04,409.20
2022-05-15,491.04,396.00
2022-06-15,0.00,491.04


In [24]:
409.2 / 31, 369.6/ 28, 409.2/31, 396.00/30,491.04/31/1.2  # let's verify the rentals 

(13.2, 13.200000000000001, 13.2, 13.2, 13.200000000000001)

User can set a vector of rental changes of life time

In [27]:
leaseStepUp = ["Lease"
                ,{"initRental": 12.0
                ,"stepUp":("byAmounts", 1, 1, -1, -3)
                ,"originTerm": 5
                ,"freq": ["DayOfMonth",15]
                ,"originDate": "2022-01-05"
                ,"status":"Current"
                ,"remainTerm": 5}
              ]
r = localAPI.runAsset("2022-01-02"
                     ,[leaseStepUp]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)

r[0]

,Balance,Rental
Date,,
2022-01-15,1498,120
2022-02-15,1095,403
2022-03-15,703,392
2022-04-15,300,403
2022-05-15,0,300


In [41]:
403 / 31, 392 / 28, 403/31, 300/30 # let's verify the rentals 

(13.0, 14.0, 13.0, 10.0)

In [45]:
leaseStepUp = ["Lease"
                ,{"initRental": 12.0
                ,"stepUp":("byAmounts", 1, 1, -1, -3)
                ,"originTerm": 5
                ,"freq": ["DayOfMonth",15]
                ,"originDate": "2022-01-05"
                ,"status":"Current"
                ,"remainTerm": 3}
              ]
r = localAPI.runAsset("2021-01-02"
                     ,[leaseStepUp]
                     ,poolAssump=("Pool"
                                    ,("Lease",None, {"Days":60}, ("byAnnualRate",0.00), ("byExtTimes",0))
                                    ,None
                                    ,None)
                     ,read=True)

r[0]

,Balance,Rental
Date,,
2022-03-15,703,392
2022-04-15,300,403
2022-05-15,0,300
